In [ ]:
# libs needed that aren't on the huggingface container 
# !pip install datasets --user
# ! pip install evaluate --user

In [ ]:
import csv
import pandas as pd
from transformers import BertTokenizerFast, TrainingArguments, Trainer, BertForSequenceClassification, \
    DataCollatorWithPadding
from datasets import Dataset, load_dataset
import csv
import pandas as pd
from sklearn import model_selection
import torch
import evaluate
import numpy as np

In [8]:
with open('Sentiment140.csv', mode='r') as file:
    tweet_data = csv.reader(file)
    
    labels = ['Polarity', 'Id', 'Date', 'Query', 'User', 'Text']
    df = pd.DataFrame(tweet_data, columns=labels)

df = df.drop(columns=['Id', 'Date', 'Query', 'User'], axis=1)
df['label'] = df['label'].replace("4", "1")
pd.to_numeric(df['label'], downcast='float')

# this is hacky & the dataset class from huggingface is supposed to be able to take in a dataframe...
# I couldn't get it to work, so I did this instead
df.to_csv("tweet_data_clean.csv", index=False, )

dataset = load_dataset('csv', data_files='tweet_data_clean.csv')
dataset = dataset['train'].train_test_split(test_size=0.2)
print(dataset)

,Polarity,Text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,4,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...


In [ ]:
def tokenize_function(example):
    """
    Tokenizes input embeddings using model tokenizer"""
    return tokenizer(example["texts"], truncation=True)

def compute_metrics(eval_preds):
    """
    Computes eval metrics"""
    metric = evaluate.load("accuracy", "f1")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# tokenize input sequences to subwords
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
tokenized_tweets = dataset.map(tokenize_function, batched=True)


# for batching
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# to load from checkpoint for eval; trainer class takes care of both training and eval.
# pretty much just a general model wrapper
model = BertForSequenceClassification.from_pretrained('results/')

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
# speeds up testing by turning off backprop objects
with torch.no_grad():
    eval_results = trainer.evaluate(tokenized_tweets["test"])

In [ ]:
print(eval_results)

In [ ]:
# to train model; see https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertForSequenceClassification
import torch
print(tokenized_tweets)
print(torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.cuda.device_count())

# init model:
# takes in a string and pulls the model from huggingface
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=30,
    per_device_eval_batch_size=30,
    num_train_epochs=3,
    weight_decay=0.01)

# train
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_tweets["train"],
    eval_dataset=tokenized_tweets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
trainer.train()
trainer.save_model()

In [ ]:
with torch.no_grad():
    eval_results = trainer.evaluate(tokenized_tweets["test"])
print(eval_results)